<a href="https://colab.research.google.com/github/nanditavenkatesh/Text-Classification-on-Twitter-data/blob/main/Text_Classification_on_Twitter_data.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

References
1. Pandas. (n.d.). pandas.read_csv — Pandas 1.2.4 Documentation. Pandas.pydata.org. https://pandas.pydata.org/docs/reference/api/pandas.read_csv.html

2. Here is how to select multiple columns in a Pandas dataframe in Python. (n.d.). Pythonhow.com. Retrieved February 2, 2024, from https://pythonhow.com/how/select-multiple-columns-in-a-pandas-dataframe/#:~:text=In%20summary%2C%20to%20select%20multiple

3. sklearn.feature_extraction.DictVectorizer. (n.d.). Scikit-Learn. Retrieved February 2, 2024, from https://scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.DictVectorizer.html#sklearn.feature_extraction.DictVectorizer.get_feature_names_out

4. PhD, E. G. (2023, November 11). Understanding Multinomial Naive Bayes Classifier. Medium. https://medium.com/@evertongomede/understanding-multinomial-naive-bayes-classifier-fdbd41b405bf

5. python - Find the item with maximum occurrences in a list. (n.d.). Stack Overflow. https://stackoverflow.com/questions/6987285/find-the-item-with-maximum-occurrences-in-a-list

6. python - How do I count the occurrences of a list item? (n.d.). Stack Overflow. https://stackoverflow.com/questions/2600191/how-do-i-count-the-occurrences-of-a-list-item

7. Logistic regression in Python with Scikit-learn. (2022, September 15). Mlnuggets. https://www.machinelearningnuggets.com/logistic-regression/

8. Tracyrenee. (2021, September 18). Conduct a Twitter Sentiment Analysis Test using Spacy. MLearning.ai. https://medium.com/mlearning-ai/conduct-a-twitter-sentiment-analysis-test-using-spacy-7715db4b2c97

9. Twitter Sentiment Analysis using Spacy. (n.d.). Kaggle.com. Retrieved February 2, 2024, from https://www.kaggle.com/code/mansimeena/twitter-sentiment-analysis-using-spacy

10. Chang, E. (2021, November 28). Complete Guide to Perform Classification of Tweets with SpaCy. Medium. https://towardsdatascience.com/complete-guide-to-perform-classification-of-tweets-with-spacy-e550ee92ca79

11. How do I get indices of N maximum values in a NumPy array? (n.d.). Stack Overflow. Retrieved February 2, 2024, from https://stackoverflow.com/questions/6910641/how-do-i-get-indices-of-n-maximum-values-in-a-numpy-array

12. Python | Remove punctuation from string. (2020, March 30). GeeksforGeeks. https://www.geeksforgeeks.org/python-remove-punctuation-from-string/#

##Part 1 - Preparing Data

In [ ]:
# copy file onto google collab from github
!wget "https://raw.githubusercontent.com/nanditavenkatesh/CSCI_6515_NLU/main/HW1/Tweets_5K.csv" "Tweets_5K.csv"

--2024-02-02 17:40:44--  https://raw.githubusercontent.com/nanditavenkatesh/CSCI_6515_NLU/main/HW1/Tweets_5K.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 455926 (445K) [text/plain]
Saving to: ‘Tweets_5K.csv’

Tweets_5K.csv       100%[===================>] 445.24K  --.-KB/s    in 0.1s    

2024-02-02 17:40:45 (3.29 MB/s) - ‘Tweets_5K.csv’ saved [455926/455926]

--2024-02-02 17:40:45--  http://tweets_5k.csv/
Resolving tweets_5k.csv (tweets_5k.csv)... failed: Name or service not known.
wget: unable to resolve host address ‘tweets_5k.csv’
FINISHED --2024-02-02 17:40:45--
Total wall clock time: 1.3s
Downloaded: 1 files, 445K in 0.1s (3.29 MB/s)


In [ ]:
#@title Imports

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import nltk
from collections import Counter
from sklearn.feature_extraction import DictVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn import metrics
from sklearn.metrics import classification_report
from sklearn.linear_model import LogisticRegression
import seaborn as sns
import spacy
import string
import re
%matplotlib inline

In [ ]:
# read csv file into a dataframe
raw_tweets = pd.read_csv("Tweets_5K.csv")
raw_tweets

,textID,text,sentiment
0,cb774db0d1,"I`d have responded, if I were going",neutral
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,negative
2,088c60f138,my boss is bullying me...,negative
3,9642c003ef,what interview! leave me alone,negative
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...",negative
...,...,...,...
4995,7dfa892961,look like i can only quick post from the dash...,neutral
4996,b60b7e3175,Thanks Elaine. I`m clipping reviews for her s...,positive
4997,efb00d5c4d,"K now that, that fiasco is all wrapped and goo...",positive
4998,65473dec6e,No. I won`t be back until the 7th,neutral


In [ ]:
# adding a column called labels with -1 for negative sentiment, 0 for neutral sentiment and 1 for positive sentiment
raw_tweets.insert(3, "labels", np.where(raw_tweets["sentiment"] == "positive",1,np.where(raw_tweets["sentiment"] == "negative",-1,0)))
raw_tweets

,textID,text,sentiment,labels
0,cb774db0d1,"I`d have responded, if I were going",neutral,0
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,negative,-1
2,088c60f138,my boss is bullying me...,negative,-1
3,9642c003ef,what interview! leave me alone,negative,-1
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...",negative,-1
...,...,...,...,...
4995,7dfa892961,look like i can only quick post from the dash...,neutral,0
4996,b60b7e3175,Thanks Elaine. I`m clipping reviews for her s...,positive,1
4997,efb00d5c4d,"K now that, that fiasco is all wrapped and goo...",positive,1
4998,65473dec6e,No. I won`t be back until the 7th,neutral,0


In [ ]:
# storing labels in a list
labels = raw_tweets["labels"].to_list()

In [ ]:
# converting the tweets into a list
text = raw_tweets['text'].to_list()

In [ ]:
# splitting each sentence and storing them as a list of words
basic_preproc_tweets = []

for sentence in text:
  basic_preproc_tweets.append(sentence.split())

In [ ]:
# counting the occurrences of each word using Counter and storing them in a dictionary
basic_preproc_tweets_counter = []

for x in basic_preproc_tweets:
  basic_preproc_tweets_counter.append(dict(Counter(x)))

In [ ]:
# matrix
v = DictVectorizer(sparse = False)
basic_preproc_bow = v.fit_transform(basic_preproc_tweets_counter)
basic_preproc_bow.shape

(5000, 15121)

In [ ]:
# laplace Smoothing
basic_preproc_bow = basic_preproc_bow + 1

In [ ]:
print("Value of X at [1460][1460]: " ,basic_preproc_bow[1460][1460])

Value of X at [1460][1460]:  1.0


In [ ]:
print("Feature of column 1460: ",v.get_feature_names_out()[1460])

Feature of column 1460:  Check:


In [ ]:
# splitting the dataset into train and test sets
X = basic_preproc_bow
y = labels
X_train, X_test, y_train, y_test = train_test_split(X, y,test_size=0.20, shuffle=False)

###Conceptual Question - Part 1

1. What are the dimensions of your feature matrix (X)?
<br> **Ans:** Here, I had stored the bag of words in the Matrix 'basic_preproc_bow'. These words were first changed to lowercase and split. Then I converted them into a Matrix using DictVectorizer. The final shape of my matrix was (5000, 15121)

---

2. What is the value of X[1460][1460]?
<br> **Ans:** The Value of X[1460][1460] is *1.0*

---

3. What does this value mean? What feature does the 1460th column represent?
<br> **Ans:** The value is the count of the number of times the word appears in the sentence. The 1460th column represnts the word 'Check:'.
---

## Part 2 - Implementing Naive Bayes

In [ ]:
#@title SkLearn Multinomial Naive Bayes
naive_bayes = MultinomialNB()
naive_bayes.fit(X_train, y_train)

#calling predict function
y_pred_nb = naive_bayes.predict(X_test)

#calculating accuracy
accuracy_nb = metrics.accuracy_score(y_test, y_pred_nb)
print("Accuracy:", accuracy_nb)

Accuracy: 0.43


In [ ]:
#@title Most Frequent Sentiment Classifier
most_frequent = max(labels, key = labels.count)
print("The Most frequent label is: ", most_frequent)

The Most frequent label is:  0


In [ ]:
# updating predicted label to 0 for all test data
y_pred_freq = np.zeros(len(y_test))

#calculating accuracy
accuracy_freq = metrics.accuracy_score(y_test, y_pred_freq)
print("Accuracy of Most Frequent Classifier:", accuracy_freq)

Accuracy of Most Frequent Classifier: 0.427


###  Conceptual Question - Part 2





####1.

Calculate whether the Naive Bayes classifier you trained would classify the following tweet as positive, neutral, or negative: “Happy birthday Annemarie”. You should do this by hand and show your work, but you can use code to get the relevant probabilities (basically, the goal of this question is to make sure that you understand how Naive Bayes works).


**Calculations done with the help of code**

In [ ]:
#@title Calculating Class priors from training set
count_labels = dict(Counter(y_train))
print("Count of each label",count_labels)

# calculating probabilities of each label

P_positive = count_labels[1]/4000
P_negative = count_labels[-1]/4000
P_neutral = count_labels[0]/4000

print("Probability of class 'Positive': ", P_positive)
print("Probability of class 'Negative': ", P_negative)
print("Probability of class 'Neutral': ", P_neutral)

Count of each label {0: 1596, -1: 1120, 1: 1284}
Probability of class 'Positive':  0.321
Probability of class 'Negative':  0.28
Probability of class 'Neutral':  0.399


In [ ]:
#@title Drop words that were not seen before
index_dict = {}
feature_list = v.get_feature_names_out()
test_Sentence = ["Happy", "birthday", "Annemarie"]
for x in test_Sentence:
  if np.where(feature_list == x)[0].size == 0:
    print(x, " is not found")
  else:
    index_dict[x] = np.where(feature_list == x)[0][0]

Annemarie  is not found


In [ ]:
index_dict

{'Happy': 2304, 'birthday': 5785}

In [ ]:
#@title Likelihoods from training for "Happy" and "birthday"
label_set = [-1,0,1]
sum_dict = {}
for label in label_set:
  label_indices = np.where(np.array(y_train) == label)[0]
  sum_row = 0
  sum_Happy = 0
  sum_birthday = 0
  for x in label_indices:
    sum_row += np.sum(X_train[x])
    sum_Happy += X_train[x][index_dict['Happy']]
    sum_birthday += X_train[x][index_dict['birthday']]
  sum_dict[label] = {"sum_row" : sum_row, "sum_Happy" : sum_Happy, "sum_birthday" : sum_birthday}

In [ ]:
sum_dict

{-1: {'sum_row': 16950578.0, 'sum_Happy': 1122.0, 'sum_birthday': 1121.0},
 0: {'sum_row': 24153184.0, 'sum_Happy': 1600.0, 'sum_birthday': 1601.0},
 1: {'sum_row': 19432106.0, 'sum_Happy': 1343.0, 'sum_birthday': 1292.0}}

In [ ]:
# probabilities calculated
prob_dict = {}
for label in label_set:
   prob_dict[label] = {'Happy': (sum_dict[label]['sum_Happy']/sum_dict[label]['sum_row']), 'birthday': (sum_dict[label]['sum_birthday']/sum_dict[label]['sum_row']) }

In [ ]:
prob_dict

{-1: {'Happy': 6.619243308399277e-05, 'birthday': 6.613343804559349e-05},
 0: {'Happy': 6.624385422642414e-05, 'birthday': 6.628525663531565e-05},
 1: {'Happy': 6.91124266201512e-05, 'birthday': 6.648790409027205e-05}}

In [ ]:
#@title Scoring test set for Positive, Negative and Neutral
P_scored_positive = prob_dict[1]['Happy'] * prob_dict[1]['birthday'] * P_positive
P_scored_negative = prob_dict[-1]['Happy'] * prob_dict[-1]['birthday'] * P_negative
P_scored_neutral = prob_dict[0]['Happy'] * prob_dict[0]['birthday'] * P_neutral

print("Positive Probability score for 'Happy birthday': ",P_scored_positive)
print("Negative Probability score for 'Happy birthday': ",P_scored_negative)
print("Neutral Probability score for 'Happy birthday': ",P_scored_neutral)

Positive Probability score for 'Happy birthday':  1.4750400660138714e-09
Negative Probability score for 'Happy birthday':  1.2257092882852522e-09
Neutral Probability score for 'Happy birthday':  1.7520053602864746e-09


**Ans**: Since the probability score for Neutral sentiment is the highest, "Happy birthday annemarie" is classified as neutral.

---

####2.
Report the model’s accuracy on the unseen test set. How does this compare to a classifier that always outputs the most frequent category in the training set (what is that classifier’s accuracy)?

<br> **Ans:** The model for Naive Bayes achieves an accuracy of 43%, while the model with Most frequent sentiment as classifier achieves an accuracy of 42.7%. Naive Bayes is only performing marginally better than baseline.

---

## Part 3 - Implementing Logistic Regression

In [ ]:
#@title Logistic Regression using skLearn
classifier = LogisticRegression(random_state = None, max_iter = 150)
classifier.fit(X_train, y_train)

# calling predict function
y_pred_lr = classifier.predict(X_test)

#calculating accuracy
accuracy_lr = metrics.accuracy_score(y_test, y_pred_lr)
print("Accuracy:", accuracy_lr)

In [ ]:
print("Classifier parameters: ",(len(classifier.coef_[0])+len(classifier.coef_[1])+len(classifier.coef_[-1])))

Classifier parameters:  45363


###Conceptual Question - Part 3

1. In class, we learned how to do binary logistic regression between 2 options. Here, there are three possible classifications, so sklearn uses a “one vs. rest” scheme where it learns a binary logistic regression model for each possible label (i.e., one logistic regression which learns to separate positive from non-positive tweets, a second that learns to separate negative from non-negative tweets, and a final logistic regression that learns to separate neutral from non-neutral tweets. How many parameters did your multiclass logistic regression model learn?
<br>**Ans:** For each classifier the number of parameters were 15121. Total number of parameters 45363.
---

2. Report the model’s accuracy on the unseen test set. How does this compare to a classifier that always outputs the most frequent category in the training set and the Naive Bayes classifier from Part 2?
<br>**Ans:** Logistic Regression model achieved an accuracy 58.3%. The model for Naive Bayes achieves an accuracy of 43%, while the model with Most frequent sentiment as classifier achieves an accuracy of 42.7%. Naive Bayes is only performing marginally better than baseline, while logistic regression is significantly better and seems to have learned some properties from the sentences.
---

## Part 4 - Implementing more elaborate pre-processing

In [ ]:
# load the english library from spacy
nlp = spacy.load("en_core_web_sm")

In [ ]:
#creating a new df for pre_processsing
pre_process_text = pd.DataFrame()
pre_process_text = raw_tweets.copy()

pre_process_text

,textID,text,sentiment,labels
0,cb774db0d1,"I`d have responded, if I were going",neutral,0
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,negative,-1
2,088c60f138,my boss is bullying me...,negative,-1
3,9642c003ef,what interview! leave me alone,negative,-1
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...",negative,-1
...,...,...,...,...
4995,7dfa892961,look like i can only quick post from the dash...,neutral,0
4996,b60b7e3175,Thanks Elaine. I`m clipping reviews for her s...,positive,1
4997,efb00d5c4d,"K now that, that fiasco is all wrapped and goo...",positive,1
4998,65473dec6e,No. I won`t be back until the 7th,neutral,0


In [ ]:
#@title Pre-process data

# Create list of stopwords from spaCy
stopwords = spacy.lang.en.stop_words.STOP_WORDS

# Creat tokenizer function
def pre_processing(sen):

  # Create token object from spacy
  tokens = nlp(sen)

  # Lemmatize each token and convert each token into lowercase
  tokens = [word.lemma_.lower().strip() for word in tokens]

  # Remove stopwords and white spaces and Remove punctuations
  tokens = [word for word in tokens if word not in stopwords and word.isalnum()]

  # change all numbers to NUM
  tokens = ['NUM' if not token.isalpha() else token for token in tokens]

  # return preprocessed list of tokens
  return tokens

# extract only tweets
pre_process_text['pre_processed_text'] = pre_process_text['text'].apply(pre_processing)


In [ ]:
pre_process_text['pre_processed_text']

0                                               [respond]
1                           [sooo, sad, miss, san, diego]
2                                           [boss, bully]
3                                      [interview, leave]
4                                     [son, release, buy]
                              ...                        
4995                 [look, like, quick, post, dashboard]
4996            [thanks, elaine, clip, review, scrapbook]
4997              [k, fiasco, wrap, good, study, midterm]
4998                                                [NUM]
4999    [sooo, tired, headache, self, inflict, course,...
Name: pre_processed_text, Length: 5000, dtype: object

In [ ]:
# converting the pre-processed tweets into a list
pre_processed_text_list = pre_process_text['pre_processed_text'].to_list()

In [ ]:
# counting the occurrences of each word using Counter and storing them in a dictionary
basic_preproc_tweets_counter_pre_processed = []

for x in pre_process_text['pre_processed_text']:
  basic_preproc_tweets_counter_pre_processed.append(dict(Counter(x)))

In [ ]:
# matrix
v_pp = DictVectorizer(sparse = False)
basic_preproc_bow_pp = v_pp.fit_transform(basic_preproc_tweets_counter_pre_processed)
basic_preproc_bow_pp.shape

(5000, 6960)

In [ ]:
# discussed with Aditya Somani
# removing top 1000 most common words
frequency_of_words = basic_preproc_bow_pp.sum(axis=0)
most_common_words_index = np.argpartition(frequency_of_words, -1000)[-1000:]
np.sort(most_common_words_index)
word_index = [i for i in range(basic_preproc_bow_pp.shape[1]) if i not in most_common_words_index]

unchanged_cols = basic_preproc_bow_pp[:, most_common_words_index]
combined_col = np.sum(basic_preproc_bow_pp[:, word_index], axis=1, keepdims=True)
#Combining top 1000 features with sum of all other features and adding laplace smoothing
updated_preproc_bow = np.hstack((unchanged_cols, combined_col)) + 1

In [ ]:
# splitting the dataset into train and test sets
X_pre_processed = updated_preproc_bow
y_pre_processed = labels
X_train_pre_processed, X_test_pre_processed, y_train_pre_processed, y_test_pre_processed = train_test_split(X_pre_processed, y_pre_processed, test_size=0.20, shuffle=False)

In [ ]:
#@title SkLearn Multinomial Naive Bayes on pre-processed tweets
naive_bayes_pp = MultinomialNB()
naive_bayes_pp.fit(X_train_pre_processed, y_train_pre_processed)

#calling predict function
y_pred_nb_pp = naive_bayes_pp.predict(X_test_pre_processed)

#calculating accuracy
accuracy_nb_pp = metrics.accuracy_score(y_test_pre_processed, y_pred_nb_pp)
print("Accuracy:", accuracy_nb_pp)

Accuracy: 0.434


In [ ]:
#@title Logistic Regression using skLearn on pre-processed tweets
classifier_pp = LogisticRegression(random_state = None, max_iter = 3000)
classifier_pp.fit(X_train_pre_processed, y_train_pre_processed)

# calling predict function
y_pred_lr_pp = classifier_pp.predict(X_test_pre_processed)

#calculating accuracy
accuracy_lr_pp = metrics.accuracy_score(y_test_pre_processed, y_pred_lr_pp)
print("Accuracy:", accuracy_lr_pp)

Accuracy: 0.633


###Conceptual Question - Part 4

1. How does this impact the performance of your Naive Bayes and Logistic Regression classification results?
<br>**Ans:** With both Naive bayes and Logistic regression, the classification results improved after using pre-processed data.

Naive Bayes achieved accuracy of 43.4% compared to 43% which was seen earlier.

Logistic Regression achieved accuracy of 63.3% compared to 58.3% which was seen earlier.


---

## Part 5

###1.
Propose and implement at least one addition to the workflow that you think will improve performance (but please still use Naive Bayes or Logistic Regression). Specifically, you can add a pre-processing step, add a feature (or class of features), etc. Explain why you chose this feature and why you think it might help performance. Report the classifier’s accuracy on the test set. Did this help as expected? Note: You will not be graded on whether your change actually improved performance.

<br> **Ans:** After Studying the article Chang, E. (2021, November 28). Complete Guide to Perform Classification of Tweets with SpaCy. Medium. https://towardsdatascience.com/complete-guide-to-perform-classification-of-tweets-with-spacy-e550ee92ca79, I thought about removing words less than 2 charactes and proper nouns to see if it would improve performance. I thought that this might improve performance since most common stop words are less than 2 characters and that proper nouns are usually not helpful in identifying the sentiment of a sentence.

After adding these steps to pre-processing I observed that the accuracy for Naive Bayes went down to 42.9%, less than Naive Bayes without pre-processing but marginally better than Most Frequent Classifier. The same goes for Logistic Regression; after these steps, I observed an accuracy of 62%, which was better than Logistic Regression without pre-processing but lesser than logistic regression with pre-processing before the additional steps were added.

In [ ]:
#creating a new df for pre_processsing
pre_process_text = pd.DataFrame()
pre_process_text = raw_tweets.copy()

pre_process_text

,textID,text,sentiment,labels
0,cb774db0d1,"I`d have responded, if I were going",neutral,0
1,549e992a42,Sooo SAD I will miss you here in San Diego!!!,negative,-1
2,088c60f138,my boss is bullying me...,negative,-1
3,9642c003ef,what interview! leave me alone,negative,-1
4,358bd9e861,"Sons of ****, why couldn`t they put them on t...",negative,-1
...,...,...,...,...
4995,7dfa892961,look like i can only quick post from the dash...,neutral,0
4996,b60b7e3175,Thanks Elaine. I`m clipping reviews for her s...,positive,1
4997,efb00d5c4d,"K now that, that fiasco is all wrapped and goo...",positive,1
4998,65473dec6e,No. I won`t be back until the 7th,neutral,0


In [ ]:
#@title Remove words less than 2 character
pre_process_text['text'] = pre_process_text['text'].apply(lambda x:' '.join([w for w in x.split() if len(w)>2]))
pre_process_text

,textID,text,sentiment,labels
0,cb774db0d1,"I`d have responded, were going",neutral,0
1,549e992a42,Sooo SAD will miss you here San Diego!!!,negative,-1
2,088c60f138,boss bullying me...,negative,-1
3,9642c003ef,what interview! leave alone,negative,-1
4,358bd9e861,"Sons ****, why couldn`t they put them the rele...",negative,-1
...,...,...,...,...
4995,7dfa892961,look like can only quick post from the dashboa...,neutral,0
4996,b60b7e3175,Thanks Elaine. I`m clipping reviews for her sc...,positive,1
4997,efb00d5c4d,"now that, that fiasco all wrapped and good too...",positive,1
4998,65473dec6e,No. won`t back until the 7th,neutral,0


In [ ]:
#@title Pre-process data with spell check
# Load the English library from SpaCy
nlp = spacy.load("en_core_web_sm")

# Create list of stopwords from spaCy
stopwords = spacy.lang.en.stop_words.STOP_WORDS

# Creat tokenizer function
def pre_processing(sen):

  # Create token object from spacy
  tokens = nlp(sen)

  # Lemmatize each token and convert each token into lowercase and removing proper nouns
  tokens = [word.lemma_.lower().strip() for word in tokens if word.pos_ != "PROPN"]

  # Remove stopwords and white spaces and Remove punctuations
  tokens = [word for word in tokens if word not in stopwords and word.isalnum()]

  # change all numbers to NUM
  tokens = ['NUM' if not token.isalpha() else token for token in tokens]

  # return preprocessed list of tokens
  return tokens

# extract only tweets
pre_process_text['pre_processed_text'] = pre_process_text['text'].apply(pre_processing)


In [ ]:
pre_process_text['pre_processed_text']

0                                             [responded]
1                                                  [miss]
2                                           [boss, bully]
3                                      [interview, leave]
4                                     [son, release, buy]
                              ...                        
4995                 [look, like, quick, post, dashboard]
4996                     [thank, clip, review, scrapbook]
4997                 [fiasco, wrap, good, study, midterm]
4998                                                [NUM]
4999    [sooo, tired, headache, self, inflict, course,...
Name: pre_processed_text, Length: 5000, dtype: object

In [ ]:
# converting the pre-processed tweets into a list
pre_processed_text_list = pre_process_text['pre_processed_text'].to_list()

In [ ]:
# counting the occurrences of each word using Counter and storing them in a dictionary
basic_preproc_tweets_counter_pre_processed = []

for x in pre_process_text['pre_processed_text']:
  basic_preproc_tweets_counter_pre_processed.append(dict(Counter(x)))

In [ ]:
# matrix
v_pp = DictVectorizer(sparse = False)
basic_preproc_bow_pp = v_pp.fit_transform(basic_preproc_tweets_counter_pre_processed)
basic_preproc_bow_pp.shape

(5000, 4964)

In [ ]:
# discussed with Aditya Somani
# removing top 1000 most common words
frequency_of_words = basic_preproc_bow_pp.sum(axis=0)
most_common_words_index = np.argpartition(frequency_of_words, -1000)[-1000:]
np.sort(most_common_words_index)
word_index = [i for i in range(basic_preproc_bow_pp.shape[1]) if i not in most_common_words_index]

unchanged_cols = basic_preproc_bow_pp[:, most_common_words_index]
combined_col = np.sum(basic_preproc_bow_pp[:, word_index], axis=1, keepdims=True)
#Combining top 1000 features with sum of all other features and adding laplace smoothing
updated_preproc_bow = np.hstack((unchanged_cols, combined_col)) + 1

In [ ]:
# splitting the dataset into train and test sets
X_pre_processed = updated_preproc_bow
y_pre_processed = labels
X_train_pre_processed, X_test_pre_processed, y_train_pre_processed, y_test_pre_processed = train_test_split(X_pre_processed, y_pre_processed, test_size=0.20, shuffle=False)

In [ ]:
#@title SkLearn Multinomial Naive Bayes on pre-processed tweets
naive_bayes_pp = MultinomialNB()
naive_bayes_pp.fit(X_train_pre_processed, y_train_pre_processed)

#calling predict function
y_pred_nb_pp = naive_bayes_pp.predict(X_test_pre_processed)

#calculating accuracy
accuracy_nb_pp = metrics.accuracy_score(y_test_pre_processed, y_pred_nb_pp)
print("Accuracy:", accuracy_nb_pp)

Accuracy: 0.429


In [ ]:
#@title Logistic Regression using skLearn on pre-processed tweets
classifier_pp = LogisticRegression(random_state = None, max_iter = 3000)
classifier_pp.fit(X_train_pre_processed, y_train_pre_processed)

# calling predict function
y_pred_lr_pp = classifier_pp.predict(X_test_pre_processed)

#calculating accuracy
accuracy_lr_pp = metrics.accuracy_score(y_test_pre_processed, y_pred_lr_pp)
print("Accuracy:", accuracy_lr_pp)

Accuracy: 0.62


###2.
Finally, take the best performing model and look at the tweets that were incorrectly categorized. Do you observe any patterns in what the model is making mistakes on? What do you think could be done to further improve results? Provide a sample of tweets  as well as their true label and their predicted label to justify your answers. Note: You do not need to know how to implement the proposed change.

<br> **Ans:** The best performing mdel was logistic regression with pre-processing. Most of the incorrect predictions are for tweets that were sarcastic, or had a lot of positive words but in a negative tone or vise versa
<br>eg:
1. {'the free fillin` app on my ipod is fun, im addicted': {'real': 0, 'predicted': -1}} - This is predicted to be negative words like addicted, but is a neutral tweet.

2. {' as much as i love to be hopeful, i reckon the chances are minimal =P i`m never gonna get my cake and stuff': {'real': -1, 'predicted': 1}} - This is the opposite of the previous case where there are a lot of positive words but the sentiment is actually negative.

3. {'2am feedings for the baby are fun when he is all smiles and coos': {'real': -1, 'predicted': 0}} - This is a sarcastic tweet interpreted as positive by the model.

Proposed change: Using vectorization, such as Word2vec, to get better meanings of words may help the model make better decisions. Using a more complex model instead of logistic regression and training on a more extensive data set with more examples of sarcastic tones would also help improve accuracy.




In [ ]:
# Tweets that were incorrectly categorized
incorrect_labels = []
for x in range (len(y_pred_lr_pp)):
  if y_pred_lr_pp[x] != y_test_pre_processed[x]:
    incorrect_labels.append({raw_tweets['text'].to_list()[x]:{"real" : y_test_pre_processed[x], "predicted": y_pred_lr_pp[x]}})
print(incorrect_labels[:10])

[{'2am feedings for the baby are fun when he is all smiles and coos': {'real': -1, 'predicted': 0}}, {' Both of you': {'real': -1, 'predicted': 0}}, {' as much as i love to be hopeful, i reckon the chances are minimal =P i`m never gonna get my cake and stuff': {'real': -1, 'predicted': 1}}, {'My Sharpie is running DANGERously low on ink': {'real': 1, 'predicted': -1}}, {'i want to go to music tonight but i lost my voice.': {'real': 1, 'predicted': 0}}, {'i`ve been sick for the past few days  and thus, my hair looks wierd.  if i didnt have a hat on it would look... http://tinyurl.com/mnf4kw': {'real': 1, 'predicted': 0}}, {'is back home now      gonna miss every one': {'real': 0, 'predicted': -1}}, {'oh Marly, I`m so sorry!!  I hope you find her soon!! <3 <3': {'real': -1, 'predicted': 0}}, {'the free fillin` app on my ipod is fun, im addicted': {'real': 0, 'predicted': -1}}, {'juss came backk from Berkeleyy ; omg its madd fun out there  havent been out there in a minute . whassqoodd ?'